In [71]:
import csv,pickle,codecs
import pandas as pd

In [72]:
fusionSeries={
    "verbes":"2015-Data/MGC-160104-03-N-ext3",
    "converts":"2015-Data/MGC-160104-derives2",
    "sorties":"2015-Data/MGC-160104-03-N-ext3-derivations"
}
sortieSeries={
    "BRASS":"-paradigmes",
    "SILVER":"-Silver" #,
#    "GOLD"
}

phonologicalMap="-N"

In [73]:
with open('2015-Data/MGC-160104-Derives2.pkl', 'rb') as input:
    derives = pickle.load(input)

In [74]:
baseDerives=derives.set_index("lexeme")["base"].to_dict()
#baseDerives

In [75]:
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [76]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [77]:
entrees={}
for element in sortieSeries:
    entrees[element]={}
    for entree in fusionSeries:
        if entree!="sorties":
            entrees[element][entree]=pd.read_csv(fusionSeries[entree]+sortieSeries[element]+".csv",sep=";",encoding="utf8")
            del entrees[element][entree][u"Unnamed: 0"]
            if "score" in entrees[element][entree].columns.tolist():
                del entrees[element][entree][u"score"]
                del entrees[element][entree][u"ajouts"]
            if "FS" in entrees[element][entree].columns.tolist():
                entrees[element][entree]["FS"]=entrees[element][entree]["FS"].apply(lambda x: recoder(x))
                entrees[element][entree]["FP"]=entrees[element][entree]["FP"].apply(lambda x: recoder(x))
    listBases=entrees[element]["verbes"]["lexeme"].tolist()
    for derive in baseDerives:
        if baseDerives[derive] in listBases:
            if entrees[element]["converts"][entrees[element]["converts"]["lexeme"]==derive]["FS"].values:
                entrees[element]["verbes"].loc[entrees[element]["verbes"]["lexeme"]==baseDerives[derive],"FS"]=entrees[element]["converts"][entrees[element]["converts"]["lexeme"]==derive]["FS"].values[0]
            if entrees[element]["converts"][entrees[element]["converts"]["lexeme"]==derive]["FP"].values:
                entrees[element]["verbes"].loc[entrees[element]["verbes"]["lexeme"]==baseDerives[derive],"FP"]=entrees[element]["converts"][entrees[element]["converts"]["lexeme"]==derive]["FP"].values[0]
    for entree in fusionSeries:
        if entree!="sorties":
            entrees[element][entree].to_csv(path_or_buf=fusionSeries["sorties"]+"-"+element+".csv",encoding="utf8",sep=";")
